In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import gzip
import matplotlib.pyplot as plt
import plotly
from scipy import stats
from scipy.optimize import curve_fit
import itertools
from collections import Counter
from Bio.Seq import Seq
import functions
import one_rep
import itertools
import function_bio_rep
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
from sklearn import preprocessing

In [3]:
global_dir = '/Users/jennysheng/Documents/Chavez_Lab/Yeast_stuff/'
fig_folder = '/Users/jennysheng/Documents/Chavez_Lab/Yeast_stuff/figures/'
columns = pd.MultiIndex.from_product([['EYFP_GLU','EYFP_GAL', '3CL_GLU', '3CL_GAL',
                                      'C145_GLU', 'C145_GAL', '3CL_GC', '3CL_GRL'],
                                     ['rep1','rep2','rep3','rep4']],
                                    names=['condition','replicate'])
raw_data = pd.read_csv(global_dir + 'Raw_yeast_growth.csv', index_col = 0)
raw_data.columns = columns

In [4]:
melt_data = pd.melt(raw_data.reset_index(),id_vars='Time')
melt_mean = melt_data.groupby(['condition', 'Time']).mean()
melt_mean = melt_mean.reset_index()
melt_mean['std'] = melt_data.groupby(['condition', 'Time']).std().reset_index()['value']

In [6]:
fig = go.Figure()
# 3CL Gal and glu
fig.add_trace(go.Scatter(x=melt_mean[melt_mean['condition'] =='3CL_GAL']['Time'],
                         y=melt_mean[melt_mean['condition'] =='3CL_GAL']['value'],
                         mode = "markers+lines",
                         name='SARS-CoV-2 3CL in Galactose',
                         marker=dict(color='#D81B60'),
                         line = dict(width = 2),
                        error_y=dict(
                        type='data', # value of error bar given in data coordinates
                        array=melt_mean[melt_mean['condition'] =='3CL_GAL']['std'],
                        visible=True)))

fig.add_trace(go.Scatter(x=melt_mean[melt_mean['condition'] =='3CL_GLU']['Time'],
                         y=melt_mean[melt_mean['condition'] =='3CL_GLU']['value'],
                         name='SARS-CoV-2 3CL in Glucose',
                         mode = "markers+lines",
                         line = dict(dash='dash', width = 4),
                         marker=dict(color='#D81B60'),
                        error_y=dict(
                        type='data', # value of error bar given in data coordinates
                        array=melt_mean[melt_mean['condition'] =='3CL_GLU']['std'],
                        visible=True)))

fig.add_trace(go.Scatter(x=melt_mean[melt_mean['condition'] =='EYFP_GAL']['Time'],
                         y=melt_mean[melt_mean['condition'] =='EYFP_GAL']['value'],
                         name='EYFP in Galactose',
                         line = dict(width = 2),
                         mode = "markers+lines",
                         marker=dict(color='#FFC107'),
                        error_y=dict(
                        type='data', # value of error bar given in data coordinates
                        array=melt_mean[melt_mean['condition'] =='EYFP_GAL']['std'],
                        visible=True)))

fig.add_trace(go.Scatter(x=melt_mean[melt_mean['condition'] =='EYFP_GLU']['Time'],
                         y=melt_mean[melt_mean['condition'] =='EYFP_GLU']['value'],
                         name='EYFP in Glucose',
                         mode = "markers+lines",
                         line = dict(dash='dash',width = 3),
                         marker=dict(color='#FFC107'),
                        error_y=dict(
                        type='data', # value of error bar given in data coordinates
                        array=melt_mean[melt_mean['condition'] =='EYFP_GLU']['std'],
                        visible=True)))

fig.add_trace(go.Scatter(x=melt_mean[melt_mean['condition'] =='C145_GAL']['Time'],
                         y=melt_mean[melt_mean['condition'] =='C145_GAL']['value'],
                         name='SARS-CoV-2 3CL C145A in Galactose',
                         line=dict(width = 2),
                         mode = "markers+lines",
                         marker=dict(color='#1E88E5'),
                        error_y=dict(
                        type='data', # value of error bar given in data coordinates
                        array=melt_mean[melt_mean['condition'] =='C145_GAL']['std'],
                        visible=True)))

fig.add_trace(go.Scatter(x=melt_mean[melt_mean['condition'] =='C145_GLU']['Time'],
                         y=melt_mean[melt_mean['condition'] =='C145_GLU']['value'],
                         name='SARS-CoV-2 3CL C145A in Glucose',
                         mode = "markers+lines",
                         line = dict(dash='dash',width = 2),
                         marker=dict(color='#1E88E5'),
                        error_y=dict(
                        type='data', # value of error bar given in data coordinates
                        array=melt_mean[melt_mean['condition'] =='C145_GLU']['std'],
                        visible=True)))

fig.add_trace(go.Scatter(x=melt_mean[melt_mean['condition'] =='3CL_GC']['Time'],
                         y=melt_mean[melt_mean['condition'] =='3CL_GC']['value'],
                         name='SARS-CoV-2 3CL in Galactose and 100um GC',
                         line = dict(width = 2),
                         mode = "markers+lines",
                         marker=dict(color='#080808'),
                        error_y=dict(
                        type='data', # value of error bar given in data coordinates
                        array=melt_mean[melt_mean['condition'] =='3CL_GC']['std'],
                        visible=True)))


fig.add_trace(go.Scatter(x=melt_mean[melt_mean['condition'] =='3CL_GRL']['Time'],
                         y=melt_mean[melt_mean['condition'] =='3CL_GRL']['value'],
                         name='SARS-CoV-2 3CL in Galactose and 10um GRL',
                         line = dict(width = 2, dash = 'dash'),
                         mode = "markers+lines",
                         marker=dict(color='#080808'),
                        error_y=dict(
                        type='data', # value of error bar given in data coordinates
                        array=melt_mean[melt_mean['condition'] =='3CL_GRL']['std'],
                        visible=True)))

fig.update_yaxes(type="log")
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False)
fig.update_layout(
    xaxis_title="Time (hours)",
    yaxis_title="OD600",
    paper_bgcolor='rgba(255,255,255,100)',
    plot_bgcolor='rgba(255,255,255,100)',
    font=dict(family="Arial",
                    size=16,
                    color="#020202"
                    )
    )
fig.show()
fig.write_image(fig_folder+"Fig1A_growth_curve.pdf")

### Bar

In [8]:
end_data = pd.read_csv(global_dir + 'Raw_data_fig_1a.csv')

In [68]:
fig = go.Figure()
no_drug = end_data.drop([6, 7])
no_drug = no_drug.drop('Avg', axis = 1)
no_drug = no_drug.set_index('Condition')
no_drug_summary = pd.DataFrame(no_drug.std(axis = 1), columns = ['std'])
no_drug_summary['mean'] = no_drug.mean(axis = 1)
no_drug_summary['variant'] = ['EYFP', '3CL', '3CL-C145A','EYFP', '3CL', '3CL-C145A']
no_drug_summary['media'] = ['glu','glu','glu','gal','gal','gal']
no_drug_gal = no_drug_summary[no_drug_summary['media']=='gal']
no_drug_glu = no_drug_summary[no_drug_summary['media']=='glu']

fig.add_trace(go.Bar(name = 'Uninduced', x = no_drug_glu['variant'], y =no_drug_glu['mean'],
                     marker_color = '#1E88E5',
                    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=no_drug_glu['std'],
            visible=True)))

fig.add_trace(go.Bar(name = 'Induced', x = no_drug_gal['variant'], y =no_drug_gal['mean'],
                     marker_color = '#D81B60',
                    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=no_drug_gal['std'],
            visible=True)))

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False, tickangle=-45)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False)
fig.update_layout(
    title={
        'text': "OD595"},
    titlefont = {'size':16},
    paper_bgcolor='rgba(255,255,255,100)',
    plot_bgcolor='rgba(255,255,255,100)',
    font=dict(family="Arial",
                    size=16,
                    color="#020202"
                    ),
    height=400,
    width = 600,
    )
fig.update_layout(barmode='group')
fig.show()
fig.write_image(fig_folder+"Fig1A_bar.pdf")

In [66]:
fig = go.Figure()
grl = end_data.loc[[4, 6]]
grl = grl.drop('Avg', axis = 1)
grl = grl.set_index('Condition')
grl_summary = pd.DataFrame(grl.std(axis = 1), columns = ['std'])
grl_summary['mean'] = grl.mean(axis = 1)
grl_summary['variant'] = ['3CL', '3CL+10uM GRL-0496']

fig.add_trace(go.Bar(x = grl_summary['variant'], y =grl_summary['mean'],
                     marker_color = '#D81B60',
                    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=grl_summary['std'],
            visible=True)))

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False, tickangle=-45)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False)
fig.update_layout(
    title={
        'text': "OD595"},
    titlefont = {'size':16},
    paper_bgcolor='rgba(255,255,255,100)',
    plot_bgcolor='rgba(255,255,255,100)',
    font=dict(family="Arial",
                    size=16,
                    color="#020202"
                    ),
    height=400,
    width = 300,
    )
fig.show()
fig.write_image(fig_folder+"Fig1B_bar.pdf")

In [67]:
fig = go.Figure()
gc = end_data.loc[[4, 7]]
gc = gc.drop('Avg', axis = 1)
gc = gc.set_index('Condition')
gc_summary = pd.DataFrame(gc.std(axis = 1), columns = ['std'])
gc_summary['mean'] = gc.mean(axis = 1)
gc_summary['variant'] = ['3CL', '3CL+100uM GC376']

fig.add_trace(go.Bar(x = gc_summary['variant'], y =gc_summary['mean'],
                     marker_color = '#D81B60',
                    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=gc_summary['std'],
            visible=True)))

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False, tickangle=-45)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False)
fig.update_layout(
    yaxis_title="OD600 (48 hours)",
    paper_bgcolor='rgba(255,255,255,100)',
    plot_bgcolor='rgba(255,255,255,100)',
    font=dict(family="Arial",
                    size=16,
                    color="#020202"
                    ),
    height=400,
    width = 300,
    )
fig.show()
fig.write_image(fig_folder+"Fig1C_bar.pdf")

In [39]:
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"colspan": 2}, None],
          [{}, {}]])

fig.add_trace(go.Bar(x = gc_summary['variant'], y =gc_summary['mean'],
                     marker_color = '#D81B60',
                    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=gc_summary['std'],
            visible=True)),row=2, col=1)


fig.add_trace(go.Bar(x = grl_summary['variant'], y =grl_summary['mean'],
                     marker_color = '#D81B60',
                    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=grl_summary['std'],
            visible=True)),row=2, col=2)


fig.add_trace(go.Bar(name = 'GAL', x = no_drug_gal['variant'], y =no_drug_gal['mean'],
                     marker_color = '#D81B60',
                    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=no_drug_gal['std'],
            visible=True)),row=1, col=1)
fig.add_trace(go.Bar(name = 'GLU', x = no_drug_glu['variant'], y =no_drug_glu['mean'],
                     marker_color = '#1E88E5',
                    error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=no_drug_glu['std'],
            visible=True)),row=1, col=1)

fig.update_layout(barmode='group', showlegend=False, xaxis_tickangle=-45)
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False, tickangle=-45)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True,
                showgrid=False)
fig.update_layout(
    paper_bgcolor='rgba(255,255,255,100)',
    plot_bgcolor='rgba(255,255,255,100)',
    height=800,
    width = 600,
    font=dict(family="Arial",
                    size=16,
                    color="#020202"
                    )
    )
fig.show()
fig.write_image(fig_folder+"Fig1A_bar.pdf")